In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import glob, os

%config InlineBackend.figure_format='retina'

In [ ]:
INSTRUMENT = 'eva' 
DRIVE_BASE_DIR = '/content/drive/My Drive/SMC 09/DDSP/eval'

fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(1, 1, 1)
legend=[]

for hyperparams in [(20,10),(20,35),(20,65),(60,10),(60,35),(60,65),(100,10),(100,35),(100,65)]:

    harmonic_distribution, noise_magnitudes = hyperparams  
    LOG_DIR = DRIVE_BASE_DIR + '/' + INSTRUMENT + \
                '_h' + str(harmonic_distribution) + \
                '_n' + str(noise_magnitudes) + \
                '_checkpoints' + \
                '/summaries/train/'
    print("Analyzing", LOG_DIR)
    legend.append('h: {}, n:{}'.format(harmonic_distribution, noise_magnitudes))

    os.chdir(LOG_DIR)
    logs=[]
    for file in glob.glob("*.v2"):
        logs.append(file)

    losses = {}
    for log_file in logs:
        for e in tf.compat.v1.train.summary_iterator(LOG_DIR+log_file):
            for v in e.summary.value:
                if v.tag == 'losses/total_loss':
                    tensor_array = tf.io.decode_raw(v.tensor.tensor_content, v.tensor.dtype)
                    losses[e.step] = tensor_array[0]
                    if e.step==20000:
                      print(tensor_array[0])
    ax.plot(list(losses.keys()), list(losses.values()))

ax.set_ylabel('total loss')
ax.set_xlabel('steps')
ax.legend(legend)
ax.grid(b=True)
_ = ax.set_ylim((5,7.5))

